In [38]:
import requests
import pandas as pd
from datetime import datetime

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "e5571f88fb32067fe934a6793b8b3108"  # Replace with your actual API key


provinces = {
    "Pathum Thani":{
        "lat": 14.0134,
        "lon": 100.5304
    },
    "Bangkok":{
            "lat": 13.7367,
            "lon": 100.5232
    },
    "Chiang Mai":{
        "lat": 18.7883,
        "lon": 98.9853
    },
    "Phuket":{
        "lat": 7.9519,
        "lon": 98.3381
    }
}
# Function to fetch and process weather data
def get_weather_data(province='Pathum Thani'):
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        timestamp = datetime.fromtimestamp(data['dt'])
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'province': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [39]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            4 non-null      datetime64[ns]
 1   year                 4 non-null      int64         
 2   month                4 non-null      int64         
 3   day                  4 non-null      int64         
 4   hour                 4 non-null      int64         
 5   minute               4 non-null      int64         
 6   province             4 non-null      object        
 7   weather_main         4 non-null      object        
 8   weather_description  4 non-null      object        
 9   main.temp            4 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 452.0+ bytes


,timestamp,year,month,day,hour,minute,province,weather_main,weather_description,main.temp
0,2025-03-27 09:02:58,2025,3,27,9,2,Pathum Thani,Clouds,few clouds,36.55
1,2025-03-27 09:00:12,2025,3,27,9,0,Pathum Wan,Clear,clear sky,36.33
2,2025-03-27 09:03:18,2025,3,27,9,3,Chiang Mai,Clouds,few clouds,35.75
3,2025-03-27 09:03:19,2025,3,27,9,3,Kathu,Clouds,few clouds,32.21


In [40]:
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

In [41]:
df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour']
)

In [43]:
from datetime import datetime
import pytz

dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)

In [44]:
print(dt)

2025-03-27 10:57:31.929321+06:42
